In [7]:
import torch 
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
import time
import torch.nn as nn
import torch.optim as optim
import numpy as np
data_dir = "/home/lbenboudiaf/Bureau/ImageClassificationWithDeepLearning/GroceryStoreDataset/dataset/dataset2"
model_name = "squeezenet"
num_classes = 81
batch_size = 8
num_epochs = 100 #Valeur Initiale par défaut comme dans le TP du Prof 
feature_extract = True
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()
    
    val_acc_history = []
    
    loss_val = []
    loss_train = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # récupéré le model et calculer sa fonction loss 
                    if is_inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase, Backward 
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            if phase == 'train':
                loss_train.append(epoch_loss)
            else:
                loss_val.append(epoch_loss)
                
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, loss_train, loss_val
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [8]:
#Definition du model Squeezenet.
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0

    if model_name == "squeezenet":
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224
    else:
        print("Invalid model name")
        exit()

    return model_ft, input_size

# Initialisé le model pour ce dataset 
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
import os
# Augmentaion et normalisation de données pour le jeux d'apprentissage.  
# Normalisation des jeux de test et de validation 
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Generation de jeux d'apprentissage et de validation 
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val', 'test']}
# Creation des dataloaders pour jeux d'apprentissage et de validation. 
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val', 'test']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Envoyer le model au GPU
model_ft = model_ft.to(device)
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observation des paramètres sont optimisé 
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Initializing Datasets and Dataloaders...
Params to learn:
	 classifier.1.weight
	 classifier.1.bias


In [9]:
import copy
import matplotlib.pyplot as plt
scratch_model,_ = initialize_model(model_name, num_classes, feature_extract=False, use_pretrained=False)
scratch_model = scratch_model.to(device)
scratch_optimizer = optim.SGD(scratch_model.parameters(), lr=0.001, momentum=0.9)
scratch_criterion = nn.CrossEntropyLoss()
_,scratch_hist, train_loss, val_loss = train_model(scratch_model, dataloaders_dict, scratch_criterion, scratch_optimizer, num_epochs=num_epochs, is_inception=(model_name=="inception"))

#ohist = []
shist = []

#ohist = [h.cpu().numpy() for h in hist]
shist = [h.cpu().numpy() for h in scratch_hist]


Epoch 0/99
----------
train Loss: 2.7503 Acc: 0.2593
val Loss: 3.2345 Acc: 0.3378

Epoch 1/99
----------
train Loss: 3.1055 Acc: 0.2716
val Loss: 2.1581 Acc: 0.2872

Epoch 2/99
----------
train Loss: 1.6519 Acc: 0.4568
val Loss: 1.2335 Acc: 0.3750

Epoch 3/99
----------
train Loss: 1.4918 Acc: 0.2593
val Loss: 1.4076 Acc: 0.3378

Epoch 4/99
----------
train Loss: 1.1598 Acc: 0.4074
val Loss: 1.1367 Acc: 0.3378

Epoch 5/99
----------
train Loss: 1.0955 Acc: 0.4321
val Loss: 1.1665 Acc: 0.3750

Epoch 6/99
----------
train Loss: 1.1299 Acc: 0.4568
val Loss: 1.6329 Acc: 0.3378

Epoch 7/99
----------
train Loss: 1.2640 Acc: 0.3827
val Loss: 1.1129 Acc: 0.3750

Epoch 8/99
----------
train Loss: 1.0626 Acc: 0.4321
val Loss: 1.6008 Acc: 0.2872

Epoch 9/99
----------
train Loss: 1.2455 Acc: 0.4321
val Loss: 1.2460 Acc: 0.3750

Epoch 10/99
----------
train Loss: 1.2240 Acc: 0.3086
val Loss: 1.1148 Acc: 0.3750

Epoch 11/99
----------
train Loss: 1.3007 Acc: 0.2222
val Loss: 1.1413 Acc: 0.3378

Ep

train Loss: 0.5045 Acc: 0.7654
val Loss: 1.9955 Acc: 0.4966

Epoch 99/99
----------
train Loss: 0.5459 Acc: 0.7284
val Loss: 1.3388 Acc: 0.5304

Training complete in 58m 15s
Best val Acc: 0.628378


In [1]:

#plt.title("Validation Accuracy vs. Number of Training Epochs")
plt.xlabel("Training Epochs")
plt.ylabel("Loss rate")
plt.plot(range(1,num_epochs+1),train_loss,label="Training Scratch", figsize=(20,10))
plt.plot(range(1,num_epochs+1),val_loss,label="Validation", figsize=(20,10))
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.show()

NameError: name 'plt' is not defined